In [1]:
import pandas as pd
import numpy as np
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
from imports import load_data, model

/Users/johnmurray/anaconda/envs/421/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 23
np.random.seed(seed)

In [3]:
train, dev, test = load_data.load_sts()

train.head()

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.


First we need to tokeinze all the words

In [4]:
max_words = 10000 #number of words to act as features
maxlen = 20 #maxium length of a sentence
embedding_dim = 100

In [5]:
tokenizer = load_data.get_tokenizer([train, dev, test], max_words)

In [6]:
word_index = tokenizer.word_index
print('Found {} unique tokens.'.format(len(word_index)))

Found 15348 unique tokens.


In [7]:
(train_x1, train_x2), (dev_x1, dev_x2), (test_x1, test_x2)=tuple(map(
                    lambda x: load_data.tokenize_df(x, tokenizer, maxlen), [train, dev, test]))

Now we can build the model

In [8]:
from keras.layers import Embedding, LSTM, Dense, Input
from keras.models import Sequential, Model
from keras import layers
import keras

First we get the pre trained embeddings from GLOVE

In [9]:
embedding_matrix = load_data.get_embedding_matrix(word_index, max_words, embedding_dim)

In [10]:
concatenated, in1, in2 = model.build_concate_input(max_words, embedding_dim, maxlen, embedding_matrix)

The output has no activation so it can output any number

In [11]:
predictions = Dense(1)(concatenated)
model_pretrain_embed = Model(inputs=[in1, in2], outputs=predictions)

In [12]:
model_pretrain_embed.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])

Add callbacks to stop training when it doesnt get better

In [13]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_mean_absolute_error',
        patience=1,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='STS_best_pretain.h5',
        monitor='val_loss',
        save_best_only=True,
    )
]

In [14]:
history = model_pretrain_embed.fit([train_x1, train_x2], train['score'], epochs=10,
            callbacks=callbacks_list,
            validation_data=([dev_x1, dev_x2], dev['score']))

Train on 5749 samples, validate on 1500 samples
Epoch 1/10
5749/5749 [==============================] - 5s 823us/step - loss: 2.1849 - mean_absolute_error: 1.2316 - val_loss: 2.3742 - val_mean_absolute_error: 1.2710
Epoch 2/10
5749/5749 [==============================] - 4s 657us/step - loss: 1.9244 - mean_absolute_error: 1.1572 - val_loss: 3.2978 - val_mean_absolute_error: 1.4774


In [15]:
model_pretrain_embed.evaluate(x=[test_x1, test_x2], y=test['score'])

1379/1379 [==============================] - 0s 167us/step


[2.6464250245070096, 1.3301014454145204]

In [16]:
model_pretrain_embed.metrics_names

['loss', 'mean_absolute_error']